In [ ]:
import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import k3d
import numpy as np
from skimage import io
from skimage.exposure import rescale_intensity
import scipy.interpolate as si
import cv2
from skimage.filters import gaussian
import ipywidgets

In [ ]:
!jupyter nbextension enable --py --user k3d

In [ ]:
def normalize8(img):
    mn = img.min()
    mx = img.max()
    mx -= mn
    img = ((img - mn)/mx) * 2**8 - 1
    return np.round(img).astype(np.uint8) 

def pad_volume(size, volume):
    size_difference = size - volume.shape
    xr, yr, zr = ((size_difference)/2).astype(int)
    xl, yl, zl = size_difference - np.array([xr, yr, zr])
    return np.pad(volume, [[xl, xr], [yl, yr], [zl, zr]])

In [ ]:
DATA = '/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/MD589/preps/CH1/thumbnail_aligned.25um'
fpath = os.path.join(DATA, '111.tif')
img = cv2.imread(fpath, cv2.IMREAD_GRAYSCALE)

In [ ]:
filepath = '/home/eddyod/.brainglobe/allen_mouse_10um_v1.2/annotation.tiff'
annotations = io.imread(filepath)

In [ ]:
SC = annotations[annotations == 302]
SC.dtype

In [ ]:
ids, counts = np.unique(SC, return_counts=True)
print(ids)
print(counts)

In [ ]:
def bspliner(cv, n=100, degree=3):
    """
    Generate a B-spline curve from a set of polygon points.

    Parameters:
    cv (array-like): Array of control vertices.
    n (int, optional): Number of points to generate along the B-spline curve. Default is 100.
    degree (int, optional): Degree of the B-spline. Default is 3.

    Returns:
    numpy.ndarray: Array of points representing the B-spline curve.
    """

    cv = np.asarray(cv)
    count = len(cv)
    degree = np.clip(degree,1,count-1)

    # Calculate knot vector
    kv = np.concatenate(([0]*degree, np.arange(count-degree+1), [count-degree]*degree))

    # Calculate query range
    u = np.linspace(False,(count-degree),n)

    # Calculate result
    return np.array(si.splev(u, (kv,cv.T,degree))).T    

def smooth_image(gray):
    # threshold
    thresh = cv2.threshold(gray, 32, 255, cv2.THRESH_BINARY)[1]
    # blur threshold image
    blur = cv2.GaussianBlur(thresh, (0,0), sigmaX=3, sigmaY=3, borderType = cv2.BORDER_DEFAULT)
    # stretch so that 255 -> 255 and 127.5 -> 0
    stretch = rescale_intensity(blur, in_range=(127.5,255), out_range=(0,255)).astype(np.uint8)
    # threshold again
    thresh2 = cv2.threshold(stretch, 0, 255, cv2.THRESH_BINARY)[1]
    # get external contour
    contours = cv2.findContours(thresh2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[0] if len(contours) == 2 else contours[1]
    big_contour = max(contours, key=cv2.contourArea)
    # draw white filled contour on black background as mas
    contour = np.zeros_like(gray)
    cv2.drawContours(contour, [big_contour], 0, 255, -1)
    # dilate mask for dark border
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (12,12))
    dilate1 = cv2.morphologyEx(contour, cv2.MORPH_CLOSE, kernel)
    # apply morphology erode
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (8,8))
    dilate2 = cv2.morphologyEx(dilate1, cv2.MORPH_ERODE, kernel)
    # blur dilate image
    blur2 = cv2.GaussianBlur(dilate2, (3,3), sigmaX=0, sigmaY=0, borderType = cv2.BORDER_DEFAULT)
    # stretch so that 255 -> 255 and 127.5 -> 0
    mask = rescale_intensity(blur2, in_range=(127.5,255), out_range=(0,255))
    #return cv2.bitwise_and(gray, gray, mask=mask.astype(np.uint8))
    return cv2.bitwise_and(gray, mask.astype(np.uint8), mask=None)
    #return dilate1


In [ ]:
arr = img.copy()
arr[arr == 255] = 0
#arr[arr == 232] = 0
plt.imshow(arr, cmap='gray')

In [ ]:
simg = img.copy()
simg[simg == 255] = 0
smooth_img = smooth_image(simg)
plt.imshow(smooth_img, cmap='gray')

In [ ]:
def merge_volumes(volumes):
    sizes = np.array([vi.shape for vi in volumes])
    volume_size = sizes.max(0) + 5
    
    volumes = [pad_volume(volume_size, vi) for vi in volumes]
    volumes = list([(v > 0).astype(np.int32) for v in volumes])
    merged_volume = np.sum(volumes, axis=0)
    #merged_volume_prob = merged_volume / float(np.max(merged_volume))
    merged_volume_prob = merged_volume / float(len(volumes))
    # increasing the STD makes the volume smoother
    # Smooth the probability
    average_volume = gaussian(merged_volume_prob, 1.0)
    color = 1
    #average_volume = merged_volume_prob
    average_volume[average_volume > 0.3] = color
    average_volume[average_volume != color] = 0
    average_volume = average_volume.astype(np.uint8)
    return average_volume

def merge_brain(volumes):
    #sizes = np.array([vi.shape for vi in volumes])
    #volume_size = sizes.max(0) + 5
    
    #volumes = [pad_volume(volume_size, vi) for vi in volumes]
    #volumes = list([(v > 0).astype(np.int32) for v in volumes])
    merged_volume = np.sum(volumes, axis=0)
    #merged_volume_prob = merged_volume / float(np.max(merged_volume))
    average_volume = merged_volume / float(len(volumes))
    average_volume = normalize8(average_volume)
    # increasing the STD makes the volume smoother
    # Smooth the probability
    #average_volume = gaussian(merged_volume_prob, 1.0)
    #color = 1
    #average_volume = merged_volume_prob
    #average_volume[average_volume > 0.3] = color
    #average_volume[average_volume != color] = 0
    #average_volume = average_volume.astype(np.uint8)
    return average_volume



In [ ]:
volumes = []
brains = ['MD585', 'MD589', 'MD594']
structure = '7n_L'
for brain in brains:
    structure_path = f'/net/birdstore/Active_Atlas_Data/data_root/atlas_data/{brain}'
    volume_path = os.path.join(structure_path, 'structure', f'{structure}.npy')
    volume = np.load(volume_path)
    volumes.append(volume)
    print(volume.shape)


In [ ]:
brains = ['MD589_25um_sagittal.tif', 'MD585_MD589.tif', 'MD594_MD589.tif']
volumes = []
fixed_path = '/net/birdstore/Active_Atlas_Data/data_root/brains_info/registration'
for brain in brains:
    brainpath = os.path.join(fixed_path, brain)
    brainimg = io.imread(brainpath)
    if brainimg.dtype == np.uint8:
        #brainimg = normalize8(brainimg)
        brainimg = brainimg.astype(np.float32)
    print(brain, brainimg.shape, brainimg.dtype)
    volumes.append(brainimg)

In [ ]:
merged_volume = merge_volumes(volumes)
print(merged_volume.dtype)

In [ ]:
# Full data structures look much better.
structure = 'IC'
structure_path = f'/net/birdstore/Active_Atlas_Data/data_root/atlas_data/MD594/structure'
volume_path = os.path.join(structure_path, f'{structure}.npy')
volume = np.load(volume_path)
print(f'1. volume dtype={volume.dtype} shape={volume.shape}')
volume = np.swapaxes(volume, 0, 2)
z_values = volume[0, ...]
ids, counts = np.unique(z_values, return_counts=True)
print(ids)
print(counts)

volume = gaussian(volume, [4,0,0])
#volume[volume > 0] = 255
z_values = volume[0, ...]
print(f'min z = {np.min(z_values)}')
print(f'max z = {np.max(z_values)}')
      
volume = volume.astype(np.uint16)
print(f'2. volume dtype={volume.dtype} shape={volume.shape}')

In [ ]:
structure = 'SC'
allen_color = 851
threshold = 0.50

In [ ]:
path = f'/net/birdstore/Active_Atlas_Data/data_root/atlas_data/AtlasV7/structure/{structure}.npy'
arr7 = np.load(path)
arr7 = np.rot90(arr7, axes=(0, 1)) 
arr7 = np.flip(arr7, axis=0)
arr7[arr7 > threshold] = allen_color
arr7[arr7 != allen_color] = 0
print(arr7.shape, arr7.dtype)

In [ ]:
path = f'/net/birdstore/Active_Atlas_Data/data_root/atlas_data/AtlasV8/structure/{structure}.npy'
arr8 = np.load(path)
#volume = gaussian(volume, 1)
#arr8[arr8 > threshold] = allen_color
#arr8[arr8 != allen_color] = 0
ids, counts = np.unique(arr8, return_counts=True)
print(ids)
print(counts)
print(arr8.shape, arr8.dtype)
#(131, 160, 186) float64

In [ ]:
animal = 'MD589'
path = f'/net/birdstore/Active_Atlas_Data/data_root/atlas_data/{animal}/structure/{structure}.npy'
animal_volume = np.load(path)
#animal_volume[animal_volume > threshold] = allen_color
#animal_volume[animal_volume != allen_color] = 0
ids, counts = np.unique(animal_volume, return_counts=True)
print(ids)
print(counts)
print(animal_volume.shape, animal_volume.dtype)

In [ ]:
volume = arr8.copy()
#volume = gaussian(volume, 1.0)
#volume = volume.astype(np.uint32)
x,y,z = np.array(volume.shape) // 2
print(x)
plt.imshow(volume[:, :, z])

In [ ]:
path = f'/net/birdstore/Active_Atlas_Data/data_root/atlas_data/AtlasV8/structure/{structure}.npy'
arr8 = np.load(path)
arr8[arr8 > 0.25] = allen_color
arr8[arr8 != allen_color] = 0
ids, counts = np.unique(arr8, return_counts=True)
print(ids)
print(counts)
print(arr8.shape, arr8.dtype)

In [ ]:
volume = arr8.copy()
x,y,z = np.array(volume.shape) // 2
print(x)
plt.imshow(volume[:, :, z])

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
z,x,y = volume.nonzero()
ax.scatter(x, y, z, zdir='y', c= 'red')
plt.show()

In [ ]:
atlas_box_scales = np.array([10,10,10])
print(atlas_box_scales)
translations = np.hstack((atlas_box_scales, 1))
print(translations.shape)
print(translations)

In [ ]:
from scipy.ndimage import zoom
def scale_axis(arr, axis, scale_factor):
    
    zoom_factors = [1] * arr.ndim  # Initialize zoom factors for all axes
    zoom_factors[axis] = scale_factor  # Set zoom factor for the specified axis
    
    return zoom(arr, zoom_factors, order=1)  # Linear interpolation

In [ ]:
print(volume.dtype, volume.shape)


In [ ]:
volume = np.array([
    [[1,2],[3,4],[5,61]],
    [[11,2],[13,4],[15,67]]
])
print(volume.dtype, volume.shape)
print(volume)
#scaled_data = scale_axis(volume, axis=1, scale_factor=2)  # Scale axis 1 by a factor of 2
print()
#print(scaled_data.dtype, scaled_data.shape)
zoomed = zoom(volume, 1.5)
print(zoomed.dtype, zoomed.shape)
print(zoomed)
